# Google Gemini Interactions API Demo

This notebook demonstrates the key features of the Google Gemini Interactions API, a unified interface for interacting with Gemini models and agents that simplifies state management, tool orchestration, and long-running tasks.

**Note:** The Interactions API is in Beta. Features and schemas are subject to breaking changes.

https://ai.google.dev/gemini-api/docs/interactions



In [ ]:
# Install python-dotenv to load .env files
! pip install python-dotenv


## 1. Setup and Installation

First, we need to install the `google-genai` package (version 1.55.0 or later) and set up the client.


In [8]:
# Install the google-genai package (uncomment if not already installed)
! pip install google-genai==1.56.0



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
from google import genai
import os

# Initialize the client
# The API key can be set via environment variable GEMINI_API_KEY in .env file
# Get your Gemini API key from https://aistudio.google.com/app/apikey
# or passed directly: client = genai.Client(api_key="YOUR_API_KEY")
client = genai.Client()

print("Client initialized successfully!")


Client initialized successfully!


## 2. Basic Interactions

The simplest way to interact with the model is by providing a text prompt. The `input` can be a string, a list containing content objects, or a list of turns with roles and content objects.


In [14]:
# Create a basic interaction with a simple text prompt
interaction = client.interactions.create(
    model="gemini-3-flash-preview",
    input="Capital of India."
)

# Display the output text
print("Model Response:")
print(interaction.outputs[-1].text)
print("\n--- Interaction Object Structure ---")
print(f"Interaction ID: {interaction.id}")
print(f"Status: {interaction.status}")
print(f"Number of outputs: {len(interaction.outputs)}")


Model Response:
The capital of India is **New Delhi**.

--- Interaction Object Structure ---
Interaction ID: v1_Chd1aXBMYVkyN0M3dUR6N0lQblpDbXlRWRIXdWlwTGFZMjdDN3VEejdJUG5aQ215UVk
Status: completed
Number of outputs: 2


## 3. Stateful Conversations

You can build **multi-turn conversations** (unlimited turns) by referencing a previous interaction using the `previous_interaction_id` parameter. This allows the server to retrieve the full context, saving you from having to resend the entire chat history. Each new turn should reference the **most recent interaction's ID** to maintain the conversation chain.


In [15]:
# First turn: Create initial interaction
input_text = "Hi, my name is Karthik. I like to play cricket.And i like watching movies."
interaction1 = client.interactions.create(
    model="gemini-3-flash-preview",
    input=input_text
)

print(f"User: {input_text}")
print(f"Model: {interaction1.outputs[-1].text}")

# Turn 2
input_text = "What is my name?"
interaction2 = client.interactions.create(
    model="gemini-3-flash-preview",
    input=input_text,
    previous_interaction_id=interaction1.id
)

print(f"User: {input_text}")
print(f"Model: {interaction2.outputs[-1].text}")




User: Hi, my name is Karthik. I like to play cricket.And i like watching movies.
Model: Hi Karthik! It’s great to meet you.

You have two excellent hobbies! Since you enjoy **cricket**, do you prefer batting, bowling, or are you an all-rounder? Also, do you have a favorite international team or player you look up to?

As for **movies**, what’s your favorite genre? Are you into big action blockbusters, thrillers, or maybe some classic drama? 

I'd love to hear more about your favorites!
User: What is my name?
Model: Your name is **Karthik**! How can I help you today?


## 4. Multi-Turn Conversations

Stateful conversations support **unlimited turns**. You can continue chaining conversations by always referencing the most recent interaction's ID. Each turn builds upon the full conversation history maintained on the server.


In [16]:
# Multi-turn conversation example (3+ turns)
# Each turn references the previous interaction to maintain context

# Turn 1: Initial interaction
input_text = "I'm planning a trip to Japan. Can you suggest 3 cities I should visit? Answer in very short sentence."
turn1 = client.interactions.create(
    model="gemini-3-flash-preview",
    input=input_text
)

print("=" * 60)
print("TURN 1")
print("=" * 60)
print(f"User: {input_text}")
print(f"Model: {turn1.outputs[-1].text}")  # Truncate for display
print(f"Interaction ID: {turn1.id}\n")

# Turn 2: Continue conversation using turn1's ID
input_text = "What's the best time of year to visit the first city you mentioned? Answer in very short sentence."
turn2 = client.interactions.create(
    model="gemini-3-flash-preview",
    input=input_text,
    previous_interaction_id=turn1.id
)

print("=" * 60)
print("TURN 2")
print("=" * 60)
print(f"User: {input_text}")
# print(f"Model: {turn2.outputs[-1].text[:200]}...")
print(f"Model: {turn2.outputs[-1].text}")
print(f"Interaction ID: {turn2.id}\n")

# Turn 3: Continue using turn2's ID (the most recent interaction)
input_text = "Can you recommend a traditional dish I should try in that city? Answer in very short sentence."
turn3 = client.interactions.create(
    model="gemini-3-flash-preview",
    input=input_text,
    previous_interaction_id=turn2.id
)

print("=" * 60)
print("TURN 3")
print("=" * 60)
print(f"User: {input_text}")
print(f"Model: {turn3.outputs[-1].text}")
print(f"Interaction ID: {turn3.id}\n")

# Turn 4: Continue using turn3's ID
input_text = "What about the second city you mentioned? What's special about it? Answer in short very short sentence." 
turn4 = client.interactions.create(
    model="gemini-3-flash-preview",
    input=input_text,
    previous_interaction_id=turn3.id
)

print("=" * 60)
print("TURN 4")
print("=" * 60)
print(f"User: {input_text}")
print(f"Model: {turn4.outputs[-1].text}")
print(f"Interaction ID: {turn4.id}\n")

print("✅ Multi-turn conversation completed!")
print(f"The model maintained context across all {4} turns without needing to resend history.")


TURN 1
User: I'm planning a trip to Japan. Can you suggest 3 cities I should visit? Answer in very short sentence.
Model: Visit Tokyo for city life, Kyoto for ancient temples, and Osaka for street food.
Interaction ID: v1_ChdfaXBMYWZxSkVQQ2h6N0lQLXNxbGtRYxIXX2lwTGFmcUpFUENoejdJUC1zcWxrUWM

TURN 2
User: What's the best time of year to visit the first city you mentioned? Answer in very short sentence.
Model: Spring and autumn are the best times.
Interaction ID: v1_ChdfaXBMYWZxSkVQQ2h6N0lQLXNxbGtRYxIXQUN0TGFmT2dMS0hWejdJUHk4YUUwQVE

TURN 3
User: Can you recommend a traditional dish I should try in that city? Answer in very short sentence.
Model: Try authentic Nigiri-zushi.
Interaction ID: v1_ChdfaXBMYWZxSkVQQ2h6N0lQLXNxbGtRYxIXQWl0TGFacV9NTzZnejdJUHdhNjQyQVU

TURN 4
User: What about the second city you mentioned? What's special about it? Answer in short very short sentence.
Model: It's famous for ancient temples and shrines.
Interaction ID: v1_ChdfaXBMYWZxSkVQQ2h6N0lQLXNxbGtRYxIXQkN0TGFZN

## 5. Retrieving Past Interactions

You can retrieve previous interactions using the interaction `id` to access past turns of the conversation.


In [19]:
# Retrieve a previous interaction using its ID
# Replace with an actual interaction ID from a previous run if needed
previous_interaction = client.interactions.get(turn4.id)

print("Retrieved Interaction:")
print(f"ID: {previous_interaction.id}")
print(f"Status: {previous_interaction.status}")
print(f"Model: {previous_interaction.model}")
print(f"\nOutputs:")
for i, output in enumerate(previous_interaction.outputs):
    print(f"  Output {i+1}: {output.text if hasattr(output, 'text') else output}")


Retrieved Interaction:
ID: v1_ChdfaXBMYWZxSkVQQ2h6N0lQLXNxbGtRYxIXQkN0TGFZNldOOU9zejdJUG1zU3F5QVE
Status: completed
Model: gemini-3-flash-preview

Outputs:
  Output 1: ThoughtContent(type='thought', signature='EpkFCpYFAXLI2nyb8h4uN+pM5dAx81FPzJuCJdAo9AknXQH8AkYUDaHNLQhf3bw9mo1MlxNLTdQskR7HS5mwNV/AVBXDuH3P3/LN5uzpx84dLTTP3PJnS4Blh5Syv34iaBXHJtQC9XAYJDthurhV4VkPH6Ei3I6j5eDZZGXY5tVPhdlqzXiWYP9+JScmf8UBqp1ZJNjglXaLBS5IpC5eUpeQk1asJzVpTTG0A9bF9EFHj/1w/7fVcHqcViGL4LRyUN8qhrjpYyQ1UaKqcoWBDMmW8xw/F34FSjwV7jOk4agZVOO5d6r2hwx4Dq6+NM0waKB2GPY83IXCXrRrA6xdRXXys79Zr8b/bW1kNxT4n7MgjP3+/WV/xnBaM8UjXFDrQUojyWpGOzlgf0nHPedtvTZEkwQttSigZrVzVHvnbbib6UeurLpDAR+DZyg8bubs5DKjJihaGotVh90SYwbDzrHj1f78fYTyrREFJmYjX9Qtt+6XUZIIc32EXQwmVFNzdPM0SzooVYZa6aTSBOutFJKLCQXalrXiMhmPVe+Yi0othjD/3QF1oVYpFNmY9lDyOh56zgi1PVZfqcc8Qno1LdHWquV9PZetV0DfdX1AJBccEHhziAiVTA1/9n5l48GJTEGnLnZO5ts363XVKOa0+W/e9mqjsceYf2QAtUyBbLeUZlRVmdwJlbiOG1TQPQz2Knh9YcRtBkxGfP14LoLkfu+xKEvJ9a6Wb2nPpiVlj/IAPdGLjXihifo39vZY0JmHMQsRx8ENA+U6EFl9zMJBkL

## 6. Stateless Conversations

You can also manage conversation history manually on the client side without using server-side state management. This gives you full control over the conversation flow.


In [21]:
# Build conversation history manually
conversation_history = [
    {
        "role": "user",
        "content": "What are the three largest cities in Spain?"
    }
]

# First interaction with the conversation history
interaction1_stateless = client.interactions.create(
    model="gemini-3-flash-preview",
    input=conversation_history
)

print("First Turn (Stateless):")
print(f"User: {conversation_history[0]['content']}")
print(f"Model: {interaction1_stateless.outputs[-1].text}\n")

# Append model response to history
conversation_history.append({
    "role": "model", 
    "content": interaction1_stateless.outputs
})

# Add next user message
conversation_history.append({
    "role": "user", 
    "content": "What is the most famous landmark in the second one?"
})

# Second interaction with full history
interaction2_stateless = client.interactions.create(
    model="gemini-3-flash-preview",
    input=conversation_history
)

print("Second Turn (Stateless):")
print(f"User: {conversation_history[-1]['content']}")
print(f"Model: {interaction2_stateless.outputs[-1].text}")
print(f"\nFull conversation history maintained on client side.")


First Turn (Stateless):
User: What are the three largest cities in Spain?
Model: The three largest cities in Spain by population are:

1.  **Madrid** (approx. 3.3 million)
2.  **Barcelona** (approx. 1.6 million)
3.  **Valencia** (approx. 800,000)

Second Turn (Stateless):
User: What is the most famous landmark in the second one?
Model: The most famous landmark in **Barcelona** is the **Sagrada Família**.

Designed by the architect Antoni Gaudí, this massive Catholic basilica has been under construction since 1882 and is world-renowned for its unique organic shapes, towering spires, and intricate facades. It is a UNESCO World Heritage site and the most visited monument in Spain.

Full conversation history maintained on client side.


## 7. Background Execution

The `background=True` parameter allows you to run interactions asynchronously, which is useful for long-running tasks or when you want to continue working while the interaction processes. Note that `background=True` requires `store=True` (which is the default).

When using background execution:
- The interaction is created immediately and returns with a status
- You can check the status periodically using `client.interactions.get(interaction_id)`
- The interaction continues processing on the server side
- Once complete, you can retrieve the final results


In [ ]:
import time

# Create an interaction with background=True
# This will run asynchronously on the server
background_interaction = client.interactions.create(
    model="gemini-3-flash-preview",
    input="Write a detailed explanation of how neural networks work, including the concepts of forward propagation, backpropagation, and gradient descent.",
    background=True  # Run in background
)

print(f"Background interaction created!")
print(f"Interaction ID: {background_interaction.id}")
print(f"Initial Status: {background_interaction.status}")
print(f"\nThe interaction is now processing in the background...")
print(f"You can continue with other tasks while it runs.\n")


In [ ]:
# Poll for completion - check the status periodically
# In a real application, you might want to do other work here
max_wait_time = 60  # Maximum time to wait (seconds)
check_interval = 2  # Check every 2 seconds
start_time = time.time()

print("Polling for completion...")
while True:
    # Retrieve the current status
    current_interaction = client.interactions.get(background_interaction.id)
    status = current_interaction.status
    
    elapsed_time = time.time() - start_time
    print(f"Status: {status} (elapsed: {elapsed_time:.1f}s)")
    
    # Check if completed
    if status == "COMPLETE":
        print("\n✅ Interaction completed!")
        print(f"\nFinal Response:")
        print(current_interaction.outputs[-1].text)
        break
    elif status in ["FAILED", "CANCELLED"]:
        print(f"\n❌ Interaction ended with status: {status}")
        break
    elif elapsed_time > max_wait_time:
        print(f"\n⏱️  Timeout reached. Interaction may still be processing.")
        print(f"Final Status: {status}")
        print(f"You can check again later using: client.interactions.get('{background_interaction.id}')")
        break
    
    time.sleep(check_interval)


## 8. Best Practices and Important Notes

### Data Storage and Retention

- **By default**, all Interaction objects are stored (`store=true`) to enable:
  - Server-side state management (with `previous_interaction_id`)
  - Background execution (using `background=true`)
  - Observability purposes

- **Retention periods:**
  - **Paid Tier**: Interactions are retained for **55 days**
  - **Free Tier**: Interactions are retained for **1 day**

- To opt out of storage, set `store=false` in your request. However, note that:
  - `store=false` is incompatible with `background=true`
  - `store=false` prevents using `previous_interaction_id` for subsequent turns

### Cache Hit Rate

Using `previous_interaction_id` to continue conversations allows the system to more easily utilize implicit caching for the conversation history, which improves performance and reduces costs.

### Mixing Interactions

You have the flexibility to mix and match Agent and Model interactions within a conversation. For instance, you can use a specialized agent (like the Deep Research agent) for initial data collection, and then use a standard Gemini model for follow-up tasks such as summarizing or reformatting, linking these steps with the `previous_interaction_id`.

### Supported Models

- `gemini-2.5-pro`
- `gemini-2.5-flash`
- `gemini-2.5-flash-lite`
- `gemini-3-pro-preview`
- `gemini-3-flash-preview`

### Limitations 

**As of:** January 2025  
**Source:** [Official API Documentation](https://ai.google.dev/gemini-api/docs/interactions)

- **Beta status**: The Interactions API is in beta/preview. Features and schemas may change.
- **Unsupported features** (coming soon):
  - Grounding with Google Maps
  - Computer Use
- **Output ordering**: Content ordering for built-in tools may sometimes be incorrect
- **Tool combinations**: Combining MCP, Function Call, and Built-in tools is not yet supported

> **Note:** Limitations are subject to change as the API evolves. Please refer to the [official documentation](https://ai.google.dev/gemini-api/docs/interactions) for the most up-to-date information.


In [ ]:
# Example: Creating an interaction with store=false to opt out of storage
# Note: This prevents using previous_interaction_id in future calls

interaction_no_store = client.interactions.create(
    model="gemini-3-flash-preview",
    input="This interaction will not be stored.",
    store=False
)

print(f"Interaction created with store=False")
print(f"ID: {interaction_no_store.id}")
print(f"Response: {interaction_no_store.outputs[-1].text}")
print("\nNote: This interaction cannot be retrieved later and cannot be used with previous_interaction_id")
